In [5]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt
import os
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

## Paths

In [6]:
train_dir = '../data/AI-CA-Data/train'
valid_dir = '../data/AI-CA-Data/valid'
test_dir = '../data/AI-CA-Data/test'
(train_dir, valid_dir, test_dir)

('../data/AI-CA-Data/train',
 '../data/AI-CA-Data/valid',
 '../data/AI-CA-Data/test')

## Data Generators (match MobileNet)

In [7]:
img_size = (224, 224)
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    rotation_range=20,
    zoom_range=0.2
)

valid_datagen = ImageDataGenerator(
    rescale=1./255
)

train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

valid_data = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

print('class_indices (ordered):', train_data.class_indices)
class_names = list(train_data.class_indices.keys())
print('class_names (ordered):', class_names)

Found 7946 images belonging to 70 classes.
Found 700 images belonging to 70 classes.
class_indices (ordered): {'Afghan': 0, 'African Wild Dog': 1, 'Airedale': 2, 'American Hairless': 3, 'American Spaniel': 4, 'Basenji': 5, 'Basset': 6, 'Beagle': 7, 'Bearded Collie': 8, 'Bermaise': 9, 'Bichon Frise': 10, 'Blenheim': 11, 'Bloodhound': 12, 'Bluetick': 13, 'Border Collie': 14, 'Borzoi': 15, 'Boston Terrier': 16, 'Boxer': 17, 'Bull Mastiff': 18, 'Bull Terrier': 19, 'Bulldog': 20, 'Cairn': 21, 'Chihuahua': 22, 'Chinese Crested': 23, 'Chow': 24, 'Clumber': 25, 'Cockapoo': 26, 'Cocker': 27, 'Collie': 28, 'Corgi': 29, 'Coyote': 30, 'Dalmation': 31, 'Dhole': 32, 'Dingo': 33, 'Doberman': 34, 'Elk Hound': 35, 'French Bulldog': 36, 'German Sheperd': 37, 'Golden Retriever': 38, 'Great Dane': 39, 'Great Perenees': 40, 'Greyhound': 41, 'Groenendael': 42, 'Irish Spaniel': 43, 'Irish Wolfhound': 44, 'Japanese Spaniel': 45, 'Komondor': 46, 'Labradoodle': 47, 'Labrador': 48, 'Lhasa': 49, 'Malinois': 50, '

## Build Model

In [8]:
num_classes = train_data.num_classes

base_model = EfficientNetB0(
    include_top=False,
    input_shape=(*img_size, 3),
    weights='imagenet'
)
base_model.trainable = False

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ efficientnetb0 (Functional)     │ (None, 7, 7, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       327,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 70)             │        17,990 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,395,497 (16.77 MB)

 Trainable params: 345,926 (1.32 MB)

 Non-trainable params: 4,049,571 (15.45 MB)

## Train (10 epochs; same callbacks)

In [9]:
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

checkpoint = ModelCheckpoint(
    filepath='../saved_models/best_model_2.h5',
    monitor='val_accuracy',
    save_best_only=True,
    verbose=1
)

history = model.fit(
    train_data,
    validation_data=valid_data,
    epochs=10,
    callbacks=[early_stop, checkpoint]
)

Epoch 1/10
249/249 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.0172 - loss: 4.2783
Epoch 1: val_accuracy improved from None to 0.01429, saving model to ../saved_models/best_model_2.h5


249/249 ━━━━━━━━━━━━━━━━━━━━ 294s 1s/step - accuracy: 0.0194 - loss: 4.2564 - val_accuracy: 0.0143 - val_loss: 4.2492
Epoch 2/10
249/249 ━━━━━━━━━━━━━━━━━━━━ 0s 816ms/step - accuracy: 0.0244 - loss: 4.2407
Epoch 2: val_accuracy did not improve from 0.01429
249/249 ━━━━━━━━━━━━━━━━━━━━ 222s 890ms/step - accuracy: 0.0218 - loss: 4.2403 - val_accuracy: 0.0143 - val_loss: 4.2519
Epoch 3/10
249/249 ━━━━━━━━━━━━━━━━━━━━ 0s 778ms/step - accuracy: 0.0202 - loss: 4.2368
Epoch 3: val_accuracy did not improve from 0.01429
249/249 ━━━━━━━━━━━━━━━━━━━━ 207s 831ms/step - accuracy: 0.0221 - loss: 4.2365 - val_accuracy: 0.0143 - val_loss: 4.2530
Epoch 4/10
249/249 ━━━━━━━━━━━━━━━━━━━━ 0s 738ms/step - accuracy: 0.0232 - loss: 4.2362
Epoch 4: val_accuracy did not improve from 0.01429
249/249 ━━━━━━━━━━━━━━━━━━━━ 198s 793ms/step - accuracy: 0.0240 - loss: 4.2336 - val_accuracy: 0.0143 - val_loss: 4.2591
